# Búsqueda de texto (*information retrieval*)
Vamos a usar el algoritmo LSI para realizar una búsqueda indexada de textos similares. Versión modificada para usar matriz TFIDF
### Cargamos librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import warnings

# Gensim
import gensim
import gensim.corpora as corpora

from gensim.models import LsiModel
warnings.filterwarnings('ignore')

# spacy para lematizar
import spacy

Utilizamos un generador para obtener los documentos del Corpus línea a línea desde el archivo del conjunto de ejemplo y convertirlos en un listado de tokens.

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
stop_words = [word.text for word in nlp.vocab if word.is_stop] #listado de stop-words

def lemmatize_doc(text, allowed_postags=['NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV']):
    """Función que devuelve el lema de una string,
    excluyendo las palabras cuyo POS_TAG no está en la lista"""
    text_out = [token.lemma_.lower() for token in nlp(text) if token.pos_ in allowed_postags and len(token.lemma_)>3]
    return text_out
            
def build_texts(fname):
    """
    Generador que devuelve el texto tokenizado a partir de un archivo
    línea a línea
    """
    with open(fname) as f:
        for line in f:
            yield lemmatize_doc(line)

In [ ]:
lee_data_file = 'lee_background.cor'

In [ ]:
texto=build_texts(lee_data_file)

In [ ]:
texto

In [ ]:
print(next(texto))

### Creamos el diccionario y el corpus para Topic Modeling
Las dos entradas para el modelo LDA son un diccionario (id2word) y un corpus de `gensim`.  

In [ ]:
class TFIDF_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector TF-IDF
    del siguiente documento en el corpus.
    El corpus es el listado de críticas alojadas en el directorio
    pasado como argumento al instanciar la clase.
    
    Procesa un documento cada vez, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, filename):
        self.filename = filename
        #creamos bigramas y trigramas
        self.bigram = gensim.models.Phrases(build_texts(self.filename), min_count=5, threshold=50) # higher threshold fewer phrases.
        #optimizamos una vez entreando
        self.bigram_mod = gensim.models.phrases.Phraser(self.bigram)

        self.trigram = gensim.models.Phrases(self.bigram_mod[build_texts(self.filename)], min_count=5, threshold=50)  
        self.trigram_mod = gensim.models.phrases.Phraser(self.trigram)
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(
            self.trigram_mod[map(lambda x: self.bigram_mod[x], build_texts(self.filename))])
        #calculamos el modelo TFIDF
        self.corpus_bow = (self.diccionario.doc2bow(text) for text in
                           self.trigram_mod[map(lambda x: self.bigram_mod[x], build_texts(self.filename))])
        self.tfidf = gensim.models.TfidfModel(self.corpus_bow)
        
    def __len__(self):
        #necesitamos saber la longitud del corpus para visualizar con pyLDAvis
        return self.diccionario.num_docs
    
    def __iter__(self):
        """
        __iter__ es un iterable => TFIDF_Corpus es un streamed iterable.
        """
        for tokens in build_texts(self.filename):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.tfidf[self.diccionario.doc2bow(self.trigram_mod[self.bigram_mod[tokens]])]

In [ ]:
# Crea diccionario
corpus_tfidf = TFIDF_Corpus(lee_data_file)



In [ ]:
# Vemos como ejemplo el primer doc
for c in corpus_tfidf:
    print(c)
    print(len(c))
    break

Recuerda que en el modelo BoW de `gensim` el primer elemento de cada tupla es el ID del término en el diccionario, y el segundo su frecuencia en el doc.  
`diccionario[ID]` devuelve el término con índice ID en el vocabulario:

In [ ]:
len(corpus_tfidf.diccionario.token2id)

In [ ]:
corpus_tfidf.tfidf.num_docs

In [ ]:
len(corpus_tfidf)

## Topic modeling

### Modelo LSI
Este modelo ordena los temas y saca un listado ordenado. Hay que especificar el número de topics.


In [ ]:
lsimodel = LsiModel(corpus=corpus_tfidf, num_topics=100, id2word=corpus_tfidf.diccionario)

In [ ]:
for c in corpus_tfidf:
    print(lsimodel[c])
    break

##  Búsqueda de documentos por temática (*information retrieval*)
Para buscar los documentos más similares a un documento dado, hay que trabajar con el modelo *space vector* generado por el algoritmo LSI. Primero, generamos una matriz LSI para todos los documentos del corpus. Para buscar el documento más parecido a un nuevo texto, calculamos su vector LSI y buscamos cuál es el más cercano dentro de la matriz LSI del corpus.

In [ ]:
#creamos un índice de similitud entre los documentos del corpus
from gensim.similarities import MatrixSimilarity

#creamos corpus transformado
lsi_corpus = lsimodel[corpus_tfidf]

In [ ]:
lsi_corpus

In [ ]:
#primer documento, como BOW_corpus no es indexable no podemos indexar tampoco lsi_corpus
for i in lsi_corpus:
    print(i)
    break

In [ ]:
#podemos recuperar los documentos que queramos en una lista con las herramientas de iteración
from itertools import islice

primer_doc = islice(lsi_corpus, 1) #devuelve un objeto de tipo generador

In [ ]:
primer_doc

In [ ]:
print(next(primer_doc)) #alternativamente list(primer_doc)[0]

In [ ]:
docs_10_15 = islice(lsi_corpus, 10, 16)

In [ ]:
for v in docs_10_15:
    print(v)

In [ ]:
#creamos índice
index = MatrixSimilarity(lsi_corpus)

El índice guarda los vectores LSI de cada texto del corpus en su atributo `index`:

In [ ]:
index.index.shape

Podemos ver la similitud de cualquier documento del corpus al resto de documentos

In [ ]:
sims = index[next(islice(lsi_corpus, 1))]
print(list(enumerate(sims)))

In [ ]:
len(sims)

In [ ]:
#nos quedamos con los 10 primeros
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims_sorted[:10])

Vemos el documento original para evaluar su parecido


In [ ]:
import linecache

linecache.getline(lee_data_file, 1) #noticia  nº0

In [ ]:
sims_sorted[1][0]

In [ ]:
linecache.getline(lee_data_file, sims_sorted[1][0]+1) #noticia más parecida

In [ ]:
linecache.getline(lee_data_file, sims_sorted[2][0]+1) #segunda noticia más parecida

También podemos calcular el documento más similar dentro del corpus a un nuevo documento calculando primero su matriz TF-IDF/BoW y luego transformando a matriz LSI

In [ ]:
new_doc = "the new Pakistan government falled in the terrorist attack by the islamic group Hamas"
texto_lemmatizado = lemmatize_doc(new_doc)

In [ ]:
texto_lemmatizado

In [ ]:
texto_new = corpus_tfidf.trigram_mod[corpus_tfidf.bigram_mod[texto_lemmatizado]]
texto_new

In [ ]:
corpus_bow_new = corpus_tfidf.diccionario.doc2bow(texto_new)
corpus_tfidf_new = corpus_tfidf.tfidf[corpus_bow_new]
lsi_corpus_new = lsimodel[corpus_tfidf_new]

In [ ]:
texto_new

In [ ]:
corpus_bow_new

In [ ]:
corpus_tfidf_new

In [ ]:
lsi_corpus_new

Ahora buscamos en el índice cuáles son los documentos más parecidos dentro del corpus al nuevo documento:

In [ ]:
sims = index[lsi_corpus_new]

In [ ]:
len(sims)

In [ ]:
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims_sorted[:10])

El texto del documento más cercano es:

In [ ]:
linecache.getline(lee_data_file, sims_sorted[0][0]+1)